# Save a file for each HUC with reach geoms and ids

To make things more cloud-friendly, we want to save a portion of the NHD database in a set of GeoJSON files which can be stored on S3. Each file will have the reach geometries and associated ids.

In [48]:
import json
import os

import psycopg2
import shapely
import shapely.wkt
import geopandas as gpd
import xarray as xr
import fsspec
import numpy as np
import pyproj
import dask.bag as db
from dask.distributed import Client

%matplotlib inline

In [2]:
def get_cursor(database):
    connection = psycopg2.connect(host="database", database=database,user="postgres", password="password")
    cursor = connection.cursor()
    return cursor

# Get all HUC12s and write a GeoJSON file for each one.

In [50]:
def get_huc12s(database):
    cursor = get_cursor(database)
    query = "select huc12 from wbdhu12"
    cursor.execute(query)
    return [c[0] for c in cursor]

In [45]:
def save_huc12_extract(huc12, out_dir):
    # get huc12 boundary
    cursor = get_cursor('nhdplushr')
    query = "SELECT wkb_geometry from wbdhu12 WHERE huc12=%s"
    cursor.execute(query, [huc12])
    huc_geom = shapely.wkb.loads(cursor.fetchone()[0].tobytes())
    
    # get reaches intersecting with huc boundary
    cursor = get_cursor('nhdplusv2')
    query = f'''
        SELECT comid, ST_Force2D(wkb_geometry) from nhdflowline WHERE ST_Intersects(
            ST_GeomFromWKB(wkb_geometry, 4326), ST_GeomFromGeoJSON(%s))
        '''
    huc_geom_str = json.dumps(shapely.geometry.mapping(huc_geom))
    cursor.execute(query, [huc_geom_str])
    reach_geoms = []
    reach_ids = []
    for reach_id, reach_geom in cursor:
        reach_ids.append(int(reach_id))
        reach_geoms.append(shapely.wkb.loads(reach_geom, hex=True))

    # make dataframe with comid and geometries and save to GeoJSON    
    df = gpd.GeoDataFrame({'comid': reach_ids + [0], 'geometry': reach_geoms + [huc_geom]})
    out_path = os.path.join(out_dir, f'{huc12}.json')
    df.to_file(out_path, driver='GeoJSON', index=False)

In [ ]:
def get_huc8s(database):
    cursor = get_cursor(database)
    query = "select huc8 from wbdhu8"
    cursor.execute(query)
    return [c[0] for c in cursor]

In [ ]:
def save_huc8_extract(huc8, out_dir):
    # get huc8 boundary
    cursor = get_cursor('nhdplushr')
    query = "SELECT wkb_geometry from wbdhu8 WHERE huc8=%s"
    cursor.execute(query, [huc8])
    ## huc_geom = shapely.wkb.loads(cursor.fetchone()[0].tobytes())
    huc_geom = shapely.wkb.loads(cursor.fetchone()[0], hex=True)

    # get reaches intersecting with huc boundary
    cursor = get_cursor('nhdplusv2')
    query = f'''
        SELECT comid, ST_Force2D(wkb_geometry) from nhdflowline WHERE ST_Intersects(
            ST_GeomFromWKB(wkb_geometry, 4326), ST_GeomFromGeoJSON(%s))
        '''
    huc_geom_str = json.dumps(shapely.geometry.mapping(huc_geom))
    cursor.execute(query, [huc_geom_str])
    reach_geoms = []
    reach_ids = []
    for reach_id, reach_geom in cursor:
        reach_ids.append(int(reach_id))
        reach_geoms.append(shapely.wkb.loads(reach_geom, hex=True))

    # make dataframe with comid and geometries and save to GeoJSON
    df = gpd.GeoDataFrame({'comid': reach_ids + [0], 'geometry': reach_geoms + [huc_geom]})
    out_path = os.path.join(out_dir, f'{huc8}.json')
    df.to_file(out_path, driver='GeoJSON', index=False)

In [51]:
def main():

    limit = npartitions = 10
    client = Client()

    ## HUC12
    huc12s = get_huc12s("nhdplushr")
    huc12_out_dir = f'/opt/data/noaa/huc12-extracts/'
    os.makedirs(huc12_out_dir, exist_ok=True)
    if len(os.listdir(huc12_out_dir)) < len(huc12s)):
        ## Only run if there appear to be missing files
        huc12_bag = db.from_sequence(huc12s, npartitions=npartitions)
        out12 = huc12_bag.map(save_huc12_extract, huc12_out_dir).compute()
    else:
        print(f"Thi")

    ## HUC8
    huc8s = get_huc8s("nhdplushr")
    huc8_out_dir = f'/opt/data/noaa/huc8-extracts/'
    os.makedirs(huc8_out_dir, exist_ok=True)
    if len(os.listdir(huc8_out_dir)) < len(huc8s)):
        huc8_bag = db.from_sequence(huc8s, npartitions=npartitions)
        out8 = huc8_bag.map(save_huc8_extract, huc8_out_dir).compute()


/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34689 instead
  warnings.warn(
